In [1]:
from sympy import fraction, Rational, Point
import numpy as np
import pandas as pd
from pprint import pprint

In [2]:
raw = open('./data/10_input.txt', 'r')
raw = raw.read().split('\n')
test = raw[:-1]
test

['.#..#..##.#...###.#............#.',
 '.....#..........##..#..#####.#..#',
 '#....#...#..#.......#...........#',
 '.#....#....#....#.#...#.#.#.#....',
 '..#..#.....#.......###.#.#.##....',
 '...#.##.###..#....#........#..#.#',
 '..#.##..#.#.#...##..........#...#',
 '..#..#.......................#..#',
 '...#..#.#...##.#...#.#..#.#......',
 '......#......#.....#.............',
 '.###..#.#..#...#..#.#.......##..#',
 '.#...#.................###......#',
 '#.#.......#..####.#..##.###.....#',
 '.#.#..#.#...##.#.#..#..##.#.#.#..',
 '##...#....#...#....##....#.#....#',
 '......#..#......#.#.....##..#.#..',
 '##.###.....#.#.###.#..#..#..###..',
 '#...........#.#..#..#..#....#....',
 '..........#.#.#..#.###...#.....#.',
 '...#.###........##..#..##........',
 '.###.....#.#.###...##.........#..',
 '#.#...##.....#.#.........#..#.###',
 '..##..##........#........#......#',
 '..####......#...#..........#.#...',
 '......##...##.#........#...##.##.',
 '.#..###...#.......#........#....#',
 '...##...#.

In [133]:
class Asteroids():

    def __init__(self, field=None):
        if len(field) > 0:
            self.field = np.array([[c for c in r] for r in field])
            self.max_cols = len(self.field[0])
            self.max_rows = len(self.field)
            self.asteroids = np.array(
                [Asteroid(c, r, self.field) for r in range(self.max_rows) \
                 for c in range(self.max_cols) \
                 if self.field[r][c] == '#']
            )
            self._has_not_been_lasered = True

    def find_best_location(self):
        visible = []
        for a in self.asteroids:
            a.scan()
            visible.append(a.asteroids_detected)
        self.best = self.asteroids[visible.index(max(visible))]
        return self.best

    def build_station(self, col, row):
        self.row = row
        self.col = col
        self.field[row][col] = '@'

    def scan_from_station(self):
        return self.scan(self.col, self.row)

    def nth_vaporized(self, n=200):
        if self._has_not_been_lasered:
            self.best.giant_laser()
            self._has_not_been_lasered = False
        return self.best.nth_vaporized(n)


In [145]:
class Asteroid():

    def __init__(self, c, r, field):
        self.col = c
        self.row = r
        self.field = field
        # self.field[self.row][self.col] = '@'
        self.max_cols = len(self.field[0])
        self.max_rows = len(self.field)
        self.neighbors = np.array(
            [(c, r) for r in range(self.max_rows) \
            for c in range(self.max_cols) \
            if self.field[r][c] == '#']
        )

    def scan(self):
        cur = (self.col, self.row)
        self.distances = [np.linalg.norm(a-cur) for a in self.neighbors]
        self.angles = np.array([np.arctan2(cur[1]-a[1], -1*(cur[0]-a[0])) for a in self.neighbors])

    def giant_laser(self):
        self._rotate()
        self._idx = np.lexsort((self.distances, self.new_angles))
        self._neighbors = [self.neighbors[i] for i in self._idx]

    def nth_vaporized(self, n):
        return self._neighbors[n-1]

    def _rotate(self):
        new_angles = []
        for a in self.angles:
            if (a <= np.pi/2) and (a >= -1 * np.pi):
                new_angles.append(np.pi/2 - a)
            else:
                new_angles.append(2.5*np.pi - a)
        self.new_angles = new_angles

    @property
    def asteroids_detected(self):
        return len(set(self.angles))
    
    @property
    def coordinates(self):
        return (self.col, self.row)

    def __str__(self):
        return f'({self.col}, {self.row})'

In [88]:
example1 = [
    '......#.#.',
    '#..#.#....',
    '..#######.',
    '.#.#.###..',
    '.#..#.....',
    '..#....#.#',
    '#..#....#.',
    '.##.#..###',
    '##...#..#.',
    '.#....####'
]

In [117]:
example2 = [
'.#..##.###...#######',
'##.############..##.',
'.#.######.########.#',
'.###.#######.####.#.',
'#####.##.#.##.###.##',
'..#####..#.#########',
'####################',
'#.####....###.#.#.##',
'##.#################',
'#####.##.###..####..',
'..######..##.#######',
'####.##.####...##..#',
'.#####..#.######.###',
'##...#.##########...',
'#.##########.#######',
'.####.#.###.###.#.##',
'....##.##.###..#####',
'.#.#.###########.###',
'#.#.#.#####.####.###',
'###.##.####.##.#..##'
]

In [128]:
asteroids = Asteroids(example1)
best = asteroids.find_best_location()

In [129]:
best.asteroids_detected

33

In [114]:
best.coordinates

(5, 8)

In [142]:
asteroids = Asteroids(example2)
asteroids.find_best_location()

In [144]:
asteroids.nth_vaporized(0)

array([11, 12])

In [25]:
asteroids.asteroids

array([[5, 0],
       [7, 0],
       [9, 0],
       [0, 1],
       [2, 1],
       [6, 1],
       [7, 1],
       [8, 1],
       [9, 1],
       [2, 2],
       [3, 2],
       [5, 2],
       [7, 2],
       [9, 2],
       [0, 3],
       [2, 3],
       [3, 3],
       [9, 3],
       [1, 4],
       [2, 4],
       [3, 4],
       [8, 4],
       [2, 5],
       [4, 5],
       [7, 5],
       [1, 6],
       [2, 6],
       [3, 6],
       [6, 6],
       [2, 7],
       [5, 7],
       [7, 7],
       [3, 8],
       [4, 8],
       [7, 8],
       [8, 8],
       [9, 8],
       [1, 9],
       [6, 9],
       [8, 9]])

In [49]:
asteroids.angles

array([-2.76108628,  3.14159265, -2.03444394, -2.24553727, -2.49809154,
       -2.03444394, -2.11121583, -2.21429744, -2.35619449, -2.55359005,
       -2.8198421 ,  3.14159265, -1.84909599, -1.9513027 , -2.15879893,
       -2.35619449, -2.67794504, -1.71269338, -1.81577499, -1.57079633,
       -1.57079633,  1.57079633, -1.44644133, -1.37340077,  0.        ,
       -1.29249667, -1.24904577, -1.10714872, -0.46364761,  0.        ,
        0.46364761, -1.21202566, -1.16590454, -0.78539816,  0.        ,
       -1.05165021, -0.46364761, -0.24497866,  0.        ,  0.24497866])

In [22]:
def asteroid_scan(field, row, col):
    
    cur = (float(row), float(col))

    mc = len(field[0])   # max columns
    mr = len(field)      # max rows
        
    asteroids = np.array([(r, c) for r in range(mr) for c in range(mc) if field[r][c] == '#'])
    
    distances = [np.linalg.norm(a-cur) for a in asteroids]
    angles = np.array([np.arctan2(a[0]-cur[0], a[1]-cur[1]) for a in asteroids])

    return asteroids, distances, angles



def asteroids_detected(field, row, col)
    
    # sorted_index = np.argsort(distances)
    # angles = angles[sorted_index]
    
    # used_angles = {}
    # detections = 0
    
    # for i, asteroid in enumerate(asteroids):
    #     k = used_angles.pop(angles[i], None)
    #     if k is None:
    #         used_angles[angles[i]] = i
    #         detections += 1
    
    # return detections

In [5]:
def best(field, very_best=False):
    result = []
    for i, row in enumerate(field):
        resrow = []
        for j, char in enumerate(row):
            if char == '.':
                resrow.append(0)
            elif char == '#':
                resrow.append(asteroids_detected(field, i, j))
        result.append(resrow)
    if very_best:
        # value = 0
        # for resrow in result:
        #     if max(resrow) > value:
        #         value = max(resrow)
        return np.max(result)
    else:
        return result